In [ ]:
#| default_exp utils.asian_1x2_pnl

In [ ]:
#| hide

from IPython.core.debugger import set_trace

%load_ext autoreload
%autoreload 2

# 1X2 and Asian Handicap Profit

> Profit for 1X2 and Asian handicap betting

In [ ]:
# |export
import numexpr
import numpy as np
import mongoengine
import pandas as pd

# Load data

Using the `config.toml` credentials included in the main repository, we must first load games data stored in our MongoDb Cluster before testing our betting environment.

In [ ]:
# | include: false

from betting_env.config.mongo import mongo_init
from betting_env.datastructure.fixtures import Fixture

In [ ]:
# | include: false

# Initialise connections.
mongo_init("prod_atlas")

# Get only one fixture.
fixture = Fixture.get_all_fixtures(1)

# Fixture.
fixture = pd.DataFrame(fixture.as_pymongo())
fixture.head()

,_id,gameId,optaGameId,gameDate,homeTeamId,homeTeamOptaId,homeTeamName,awayTeamId,awayTeamOptaId,awayTeamName,...,awayTeamLineupSlots,awayTeamFormation,preGameOdds1,preGameOddsX,preGameOdds2,preGameAhHome,preGameAhAway,lineId,result,postGameGd
0,63fbf2a515a225c64d74249c,174dba7291174b4dbbfa9ea12dd944bb45bdd8ed905524...,990997,2018-08-22T18:45:00.000Z,126905d14981e6b97912ad4fec354035ccef26cb8ec4e1...,7,Aston Villa,419088133137a53bfdb1b7e2e682d223d33a6fa075bbfe...,94,Brentford,...,"[1, 5, 9, 8, 3, 6, 7, 10, 4, 11, 2]",4-2-3-1,2.62,3.53,2.77,1.86,2.06,0.0,1.0,0


# 1X2 and Asian Handicap Market

There are many other types of bets available in this area of sports betting, including the most well-known 1X2. However, the latter presupposes that the investor has a 33% chance of winning and must wager on one of the three possible outcomes: the <b>Home Team</b>, The <b>Away Team</b>, or a <b>Draw</b>.

Due to this, a brand-new sort of betting type called Asian handicaps emerged, which denotes that one team effectively leads the other side. It eliminates the possibility of a draw in a game, reducing the number of outcomes to two and raising the likelihood of winning to 50%.

We have to note that the favorite team that is at a disadvantage is denoted with a negative sign (-).
A plus sign (+) is used to indicate the handicap advantage enjoyed by the underdog squad.

In this regard, we have 3 different lines which are available for different games : 

   - <b> Half Integer Lines :</b> These are the lines that perform the best when it comes to computing payoffs for outcomes with binary options. The outcome of the wager is either a win or a loss.
    
   - <b> Integer Lines :</b> If the handicap chosen is equal to the score difference, it can result in a bet being returned.
    
   - <b> Quarter Integer Lines :</b> Divide your wager equally between the two AH lines above. For instance, a handicap of -0.75 is really two handicaps for the price of one, with half of your bet being made at -0.5 and the other half at -1, both of which are given the same odds. In this case, we have a win, a partial win, a partial loss, and a whole loss.

# PnL computation

## 1X2 market

The PnL of the 1x2 market depends on the match outcome (home-win/draw/away-win) and is *binary*: the punter can either make a profit and get his stake returned or loose his stake.

$$ profit_{1X2} =  (Odds \times Stake) - Stake$$
With:

+ Stake : Invested money
+ Odds : 1X2 betting odds

In [ ]:
# | export


def pnl_1X2(
    bet: list,  # The betting list (5 ints) for the provided action.
    balance: float,  # Current account balance.
    results: np.ndarray,  # Game result (Binary side outcome), shape=(1,5).
    odds: np.ndarray,  # odds for the current game, shape=(1,5)
) -> float: # 1X2 PnL
    "Returns the 1X2 PnL."
    # reward(could be positive or negative)
    reward = numexpr.evaluate("sum(bet * balance * results * odds)")
    # Invested money
    expense = numexpr.evaluate("sum(bet * balance)")
    # return pnl
    return reward - expense

In [ ]:
# | include: false

# Balance.
balance = 100
# Bet action (here, our action is betting 20% of our balance on away team).
bet = [0, 0, 0.2, 0, 0]
# Game odds.
odds = fixture[
    [
        "preGameOdds1",
        "preGameOdds2",
        "preGameOddsX",
        "preGameAhHome",
        "preGameAhAway",
    ]
].values
# Game result index(homewin -> 0 , draw -> 1, awaywin -> 2).
game_result = fixture["result"].values
binary_result = np.zeros(shape=(1, odds.shape[1]))
binary_result[
    np.arange(binary_result.shape[0], dtype=np.int32),
    np.array([game_result], dtype=np.int32),
] = 1
# 1X2 Pnl.
profit_1x2 = pnl_1X2(
    bet=bet,
    balance=balance,
    results=binary_result,
    odds=odds,
)
print("Balance:", balance)
print("Bet action:", bet)
print("Odds:", odds)
print("Game result:", binary_result)
print("1X2 PnL:", profit_1x2)

Balance: 100
Bet action: [0, 0, 0.2, 0, 0]
Odds: [[2.62 2.77 3.53 1.86 2.06]]
Game result: [[0. 1. 0. 0. 0.]]
1X2 PnL: -20.0


## Asian handicap market

The PnL for the Asian handicap line is more complicated. It is related to the game goal-difference and can result, as we decribed above, in 5 outcomes: win/half-win/fold/half-loss/loss.

$$ profit_{AH} = \begin{cases} 
     0.0 & \text{If Gd = 0} \\\\ 
     odds -1 & \text{If Gd >= 0.5} \\\\
     -1.0 & \text{If Gd <= -0.5} \\\\
     \frac{odds -1}{2}& \text{If Gd = 0.25}\\\\
     -0.5 & \text{If Gd = -0.25} \\\\
 0.0 & \text{Otherwise} \end{cases} $$
 
 With:

+ Gd : goal difference advantage
+ Odds : 1X2 betting odds

In [ ]:
# | export


def pnl_ah(
    bet: list,  # The betting list (5 ints) for the provided action.
    balance,  # Current account balance.
    obs_gd: int,  # Game goal-difference.
    ah_line: float,  # Asian line could be integer, half or quarter line.
    ah_odds: float,  # Asian handicap odds for the given side.
) -> float:  # Asian Handicap PnL.
    "Returns the Asian Handicap PnL"
    def _pnl_ah(obs_gd, ah_line, ah_odds):
        "provides the asian outcome given for a unit bet."
        # Team advantage.
        gd_advantage = obs_gd + ah_line

        # Deal with all advantage cases.
        # 0 advantage.
        if gd_advantage == 0:
            return 0.0
        # Positive adv and higher than 0.5.
        # Win the bet.
        elif gd_advantage >= 0.5:
            return ah_odds - 1
        # Negative adv and < than -0.5.
        # Lose the bet.
        elif gd_advantage <= -0.5:
            return -1.0

        # Positive adv and equal to 0.25.
        # Win the half.
        elif gd_advantage == 0.25:
            return (ah_odds - 1) * 0.5

        # Positive adv and equal to -0.25.
        # Lose the half stake of your bets.
        elif gd_advantage == -0.25:
            return -0.5
        # Can't compute Asian Pnl for the observed handicap.
        else:
            return 0.0

    # Calculate profit for a unit bet.
    profit = _pnl_ah(obs_gd, ah_line, ah_odds)
    # Return pnl for the invested amount.
    return numexpr.evaluate("sum(bet * balance * profit)").item()

In [ ]:
# | include: false

# Bet action (here, our action is betting 70% of our balance on away team).
bet = [0, 0, 0, 0.7, 0]

# Game goal diff.
gd = fixture["postGameGd"].values[0]
# Asian Handicap line.
ah_line = fixture["lineId"].values[0]
# Asian Handicap home win odds.
ah_odds = odds[:, 3:4].item()
# Asian Handicap Pnl.
profit_ah = pnl_ah(
    bet=bet, balance=balance, obs_gd=gd, ah_line=ah_line, ah_odds=ah_odds
)

print("Balance:", balance)
print("Bet action:", bet)
print("Game Goal difference:", gd)
print("Asian Handicap Line:", ah_line)
print("Asian Handicap odd:", ah_odds)
print("Asian Handicap PnL:", profit_ah)

Balance: 100
Bet action: [0, 0, 0, 0.7, 0]
Game Goal difference: 0
Asian Handicap Line: 0.0
Asian Handicap odd: 1.86
Asian Handicap PnL: 0.0


In [ ]:
#| hide

import nbdev

nbdev.nbdev_export()